In [1]:
import numpy as np
import pandas as pd


def f_first(X):
    return 100 * (X[1] - X[0] ** 2) ** 2 + (1 - X[0]) ** 2


dfirst = [
    lambda X: 2 * (200 * X[0] ** 3 - 200 * X[0] * X[1] + X[0] - 1),
    lambda X: 200 * (X[1] - X[0] ** 2)
]


def df_first(X, i):
    return dfirst[i](X)


def f_second(X):
    return (X[1] - X[0] ** 2) ** 2 + (1 - X[0]) ** 2


dsecond = [
    lambda X: 2 * (2 * X[0] ** 3 - 2 * X[0] * X[1] + X[0] - 1),
    lambda X: 2 * (X[1] - X[0] ** 2)
]


def df_second(X, i):
    return dsecond[i](X)


def f_third(X):
    return (1.5 - X[0] * (1 - X[1])) ** 2 + (2.25 - X[0] * (1 - X[1] ** 2)) ** 2 + (2.625 - X[0] * (1 - X[1] ** 3)) ** 2


dthird = [
    lambda X: 2 * X[0] * (X[1] ** 6 + X[1] ** 4 - 2 * X[1] ** 3 - X[1] ** 2 - 2 * X[1] + 3) + 5.25 * X[1] ** 3 + 4.5 *
              X[1] ** 2 + 3 * X[1] - 12.75,
    lambda X: X[0] * (X[0] * (6 * X[1] ** 5 + 4 * X[1] ** 3 - 6 * X[1] ** 2 - 2 * X[1] - 2) + 15.75 * X[1] ** 2 + 9 * X[
        1] + 3)
]


def df_third(X, i):
    return dthird[i](X)


def f_fourth(X):
    return (X[0] + X[1]) ** 2 + 5 * (X[2] - X[3]) ** 2 + (X[1] - 2 * X[2]) ** 4 + 10 * (X[0] - X[3]) ** 4


dfourth = [
    lambda X: 2 * (X[0] + X[1]) + 40 * (X[0] - X[3]) ** 3,
    lambda X: 2 * (X[0] + X[1]) + 4 * (X[1] - 2 * X[2]) ** 3,
    lambda X: 10 * (X[2] - X[3]) - 6 * (X[1] - 2 * X[2]) ** 3,
    lambda X: -10 * (X[2] - X[3]) - 40 * (X[0] - X[3]) ** 3
]


def df_fourth(X, i):
    return dfourth[i](X)


BOUNDS = 10

equations = {
    '100(x_2−x_1^2)^2+(1-x_1)^2': [f_first, df_first, np.random.uniform(-BOUNDS, BOUNDS, 2)],
    '(x_2−x_1^2)^2+(1-x_1)^2': [f_second, df_second, np.random.uniform(-BOUNDS, BOUNDS, 2)],
    '(1.5-x_1*(1-x_2))^2+(2.25-x_1*(1-x_2^2))^2+(2.625-x_1*(1 -x_2^3))^2': [f_third, df_third,
                                                                            np.random.uniform(-BOUNDS, BOUNDS, 2)],
    '(x_1+x_2)^2+5(x_3−x_4)^2+(x_2−2x_3)^4+10(x_1−x_4)^4': [f_fourth, df_fourth, np.random.uniform(-BOUNDS, BOUNDS, 4)]
}


In [2]:

def secant_method(x1,x2,df_x1,df_x2):
    a = (df_x2 - df_x1)/(x2 - x1)
    b = df_x1 - x1 * a
    return -b/a


def brent(f,df,start,end,eps=1e-5):
    u = x = w = v = 0.5 * (start + end)
    f_u = f_x = f_w = f_v = f(x)
    df_u = df_x = df_w = df_v = df(x)
    d = e = end - start

    function_calls_number = 1
    step_details = []

    while True:
        step_details.append([start, end, x, f_x, df_x, w, f_w, df_w, v, f_v, df_v, d, e, u, f_u, df_u])
        g,e = e,d
        candidates = []

        if x != w and df_x != df_w:
            u1 = secant_method(x,w,df_x,df_w)
            if start + eps <= u1 <= end - eps and np.abs(u1 - x) < 0.5 * g:
                candidates.append(u1)

        if x != v and df_x != df_v:
            u2 = secant_method(x,v,df_x,df_v)
            if start + eps <= u2 <= end - eps and np.abs(u2 - x) < 0.5 * g:
                candidates.append(u2)

        if len(candidates) == 2:
            u = candidates[0]
        else:
            if df_x > 0:
                u = 0.5 * (start + x)
            else:
                u = 0.5 * (x + end)

        if np.abs(u - x) < eps:
            u = x + np.sign(u - x) * eps

        d = np.abs(x - u)
        f_u = f(u)
        df_u = df(u)
        function_calls_number += 1
        if f_u <= f_x:
            if u >= x:
                start = x
            else:
                end = x
            v,w,x = w,x,u
            f_v,f_w,f_x = f_w,f_x,f_u
            df_v,df_w,df_x = df_w,df_x,df_u
        else:
            if u >= x:
                end = u
            else:
                start = u
            if f_u <= f_w or w == x:
                v,w = w,u
                f_v,f_w = f_w,f_u
                df_v,df_w = df_w,df_u
            elif f_u <= f_v or v == x or v == w:
                v = u
                f_v = f_u
                df_v = df_u

            if abs(d - g) > eps and end - start != 0:
                break

    step_details.append([start, end, x, f_x, df_x, w, f_w, df_w, v, f_v, df_v, d, e])
    return x, function_calls_number, step_details


In [3]:

def golden_ratio(function,start,end,epsilon=1e-5):
    phi = 0.5 * (3 - np.sqrt(5))

    x1 = start + (end - start)*phi
    x2 = end - (end - start)*phi

    x1_value = function(x1)
    x2_value = function(x2)

    while end - start > epsilon:
        if x1_value > x2_value:
            start = x1
            x1 = x2
            x2 = end + start - x1
            x1_value = x2_value
            x2_value = function(x2)
        else:
            end = x2
            x2 = x1
            x1 = start + end - x2
            x2_value = x1_value
            x1_value = function(x1)

    return 0.5 * (x1 + x2)


def coordinate_descent(X0,f,df,eps=1e-5,max_iter=5e4):
    step_details = []
    args_count = len(X0)
    k = 0

    X = X0.copy()
    step_details.append([X, None, f(X)])

    while k < max_iter:
        i = k % args_count
        g = np.zeros(shape=(args_count, ), dtype=float)
        g[i] = df(X, i)
        f_next_X = lambda a: X - a * g
        alpha = golden_ratio(lambda a: f(f_next_X(a)), -BOUNDS, BOUNDS, epsilon=eps)
        next_X = f_next_X(alpha)

        step_details.append([next_X, X, f(next_X)])
        prev_X, X = X, next_X
        k += 1

        if np.sum(np.abs(step_details[-1][-1] - step_details[-2][-1])) < eps:
            break

    return step_details


In [4]:

def steepest_descent(X0,f,df,eps=1e-5,max_iter=5e4):
    step_details = []
    args_count = len(X0)
    k = 0

    X = X0.copy()
    step_details.append([X, None, f(X)])

    while k < max_iter:
        g = np.array([df(X, i) for i in range(args_count)])
        f_next_X = lambda a: X - a * g
        alpha = golden_ratio(lambda a: f(f_next_X(a)), -BOUNDS, BOUNDS, epsilon=eps)
        next_X = f_next_X(alpha)

        step_details.append([next_X, X, f(next_X)])
        prev_X, X = X, next_X
        k += 1

        if np.sum(np.abs(step_details[-1][-1] - step_details[-2][-1])) < eps:
            break

    return step_details


In [5]:

gradient_descents = {
    'coordinate': coordinate_descent,
    'steepest': steepest_descent,
}

for descent in gradient_descents:
    for equation in equations:
        f, df, X0 = equations[equation]
        steps = gradient_descents[descent](X0, f, df)
        print(f'{descent}, min of {equation} is {steps[-1][0]} with value {steps[-1][-1]}')


coordinate, min of 100(x_2−x_1^2)^2+(1-x_1)^2 is [0.51746481 0.26776982] with value 0.2328402069588744
coordinate, min of (x_2−x_1^2)^2+(1-x_1)^2 is [1.00583723 1.01170852] with value 3.407323281034863e-05
coordinate, min of (1.5-x_1*(1-x_2))^2+(2.25-x_1*(1-x_2^2))^2+(2.625-x_1*(1 -x_2^3))^2 is [2.97838463 0.49459198] with value 7.660825336896408e-05
coordinate, min of (x_1+x_2)^2+5(x_3−x_4)^2+(x_2−2x_3)^4+10(x_1−x_4)^4 is [0.00092382 0.04920516 0.14651969 0.13676129] with value 0.009928654406792602
steepest, min of 100(x_2−x_1^2)^2+(1-x_1)^2 is [0.99774712 0.99549588] with value 5.076642150446819e-06
steepest, min of (x_2−x_1^2)^2+(1-x_1)^2 is [1.00304718 1.00749586] with value 1.1223559044351373e-05
steepest, min of (1.5-x_1*(1-x_2))^2+(2.25-x_1*(1-x_2^2))^2+(2.625-x_1*(1 -x_2^3))^2 is [6.12603269 0.80797948] with value 0.19269548018060215
steepest, min of (x_1+x_2)^2+5(x_3−x_4)^2+(x_2−2x_3)^4+10(x_1−x_4)^4 is [ 9.40304550e-04 -5.59811784e-05  3.14630562e-02  3.10857938e-02] with val